In [19]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [20]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [21]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [22]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [23]:
bnf_pandas = pd.read_csv("../data/midcp_1hr_tv_new.csv")

In [42]:
# symbol = 'bnf'
# symbol = 'nifty'
# symbol = 'fnf'
symbol = 'midcp'
# symbol = 'sensex'
# symbol = 'bankex'

if symbol == 'bnf' or symbol == 'bankex':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.0001
elif symbol == 'nifty' or symbol == 'fnf':
    LEVERAGE_ = 7
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.0002
elif symbol == 'midcp':
    LEVERAGE_ = 12
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.0005
elif symbol == 'sensex':
    LEVERAGE_ = 8
    LOT_SIZE_ = 10
    SLIPPAGE_ = 0.0001

PORTFOLIO = 1_00_00_000

In [25]:
bnf_pandas.tail()

,time,open,high,low,close
4702,2024-09-27T11:15:00+05:30,13342.5000,13350.1000,13318.7000,13341.9000
4703,2024-09-27T12:15:00+05:30,13341.6000,13347.0500,13315.5000,13325.1000
4704,2024-09-27T13:15:00+05:30,13324.8500,13357.3500,13307.7500,13330.0000
4705,2024-09-27T14:15:00+05:30,13329.9000,13356.3500,13317.4000,13333.2500
4706,2024-09-27T15:15:00+05:30,13333.1500,13337.3000,13319.0000,13325.4500


In [26]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas

In [27]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [28]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

In [29]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


In [30]:
def macd_logic_from_df(df, fast_ma, slow_ma):
    
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    close_prices = df['close']
    
    fast_ema = close_prices.ewm(span=fast_ma, adjust=False).mean()
    slow_ema = close_prices.ewm(span=slow_ma, adjust=False).mean()

    macd = slow_ema - fast_ema
    # prev_macd = macd.shift(1)
    
    df['macd'] = macd
    # df['prev_macd'] = prev_macd
    
    return df

In [31]:
df = macd_logic_from_df(bnf_pandas, 26, 12)
df.tail()

,time,open,high,low,close,index,datetime,macd
4702,2024-09-27T11:15:00+05:30,13342.5000,13350.1000,13318.7000,13341.9000,2024-09-27 11:15:00,2024-09-27 11:15:00,27.6820
4703,2024-09-27T12:15:00+05:30,13341.6000,13347.0500,13315.5000,13325.1000,2024-09-27 12:15:00,2024-09-27 12:15:00,30.0874
4704,2024-09-27T13:15:00+05:30,13324.8500,13357.3500,13307.7500,13330.0000,2024-09-27 13:15:00,2024-09-27 13:15:00,32.0200
4705,2024-09-27T14:15:00+05:30,13329.9000,13356.3500,13317.4000,13333.2500,2024-09-27 14:15:00,2024-09-27 14:15:00,33.4285
4706,2024-09-27T15:15:00+05:30,13333.1500,13337.3000,13319.0000,13325.4500,2024-09-27 15:15:00,2024-09-27 15:15:00,33.5289


In [32]:
# Positional

def execute(df, sl_pct):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    remark = ""

    for i in range(1, len(df)):
        points = 0
        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]
        current_macd = df.iloc[i]['macd']
        current_ma = df.iloc[i]['ma']

        if not in_trade:
            if df.iloc[i-1]['macd'] > 0:
                # Previous Candle has a signal
                signal_entry_price = df.iloc[i-1]['high']
                signal_initial_sl = signal_entry_price * (1 - sl_pct)
                signal_generation_time = df.iloc[i-1]['datetime']

                if current_candle_high >= signal_entry_price:
                    if current_candle_open >= signal_entry_price:
                        if current_candle_low <= signal_entry_price:
                            # Entry Triggered
                            in_trade = True
                            entry_time = df.iloc[i]['datetime']
                            entry_price = signal_entry_price
                            points = 0
                        else:
                            # Trade Skipped, Gap Open Outside Entry
                            continue
                    else:
                        # Entry Triggered
                        in_trade = True
                        entry_time = df.iloc[i]['datetime']
                        entry_price = signal_entry_price
                        points = 0
                else:
                    # Check Next Iteration for Better Candle Scenario
                    continue

        if in_trade:
            trade_entry_price = signal_entry_price
            trade_initial_sl = signal_initial_sl
            trade_final_sl = signal_initial_sl
            
            if (
                not is_trailing_active
                and current_candle_low > current_ma
            ):
                is_trailing_active = True

            if not is_trailing_active:
                if current_candle_open <= signal_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        # Check if this Gap Open below SL candle is the Entry Candle
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # Initial SL Hit
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

                elif df.iloc[i]['macd'] < 0:
                    # MACD Reversal
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "MACD Reversal"

            else:
                trade_final_sl = max(trade_initial_sl, current_ma)

                if current_candle_open <= trade_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # Despite Trailing, Initial SL hit
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL hit"

                elif current_candle_close <= trade_final_sl:
                    # Price Closed below TSL
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    is_trailing_active = False
                    remark = "TSL Hit"

                elif df.iloc[i]['macd'] < 0:
                    # MACD Reversal
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "MACD Reversal"

            if points:
                qty = int(round(PORTFOLIO * LEVERAGE_ / entry_price / LOT_SIZE_)) * LOT_SIZE_
                slippage = SLIPPAGE_ * (entry_price + exit_price)
                # slippage = 10
                final_points = points - slippage
                # final_points = points
                trade = {
                    "Signal Generated At": signal_generation_time,
                    "Trade Type": "LONG",
                    "Entry Time": entry_time,
                    "Entry Price": entry_price,
                    "Initial SL": trade_initial_sl,
                    "Final SL": trade_final_sl,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    "Qty": qty,
                    "Leverage": LEVERAGE_,
                    "ROI%": (final_points * qty / PORTFOLIO) * 100,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                }
                # print(trade)
                trade_book.append(trade)
                points = 0
                in_trade = False
                already_signal_exists = False
                remark = ""
                is_trailing_active = False

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [33]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [43]:
stats_dictionary = {}

slow_ma = 10
fast_ma = 5
trailing_ma = 80
sl_pct = 0.0066
# df1 = resample(pl.DataFrame(bnf_pandas), '60m', '15m')
# df2 = df1.to_pandas()
df = macd_logic_from_df(bnf_pandas, slow_ma, fast_ma)
df['ma'] = df['close'].rolling(window=trailing_ma).mean()
tb = execute(df, sl_pct)
variation = f'MA1 : {slow_ma}, MA2 : {fast_ma}, SL : {sl_pct * 100}%'
stats = generate_stats(tb, variation)

In [44]:
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                      Variation
2017       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  MA1 : 10, MA2 : 5, SL : 0.66%
2018       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  MA1 : 10, MA2 : 5, SL : 0.66%
2019       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  MA1 : 10, MA2 : 5, SL : 0.66%
2020       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  MA1 : 10, MA2 : 5, SL : 0.66%
2021       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  MA1 : 10, MA2 : 5, SL : 0.66%
2022     203.0898           64  35.9375               21.8715             -7.3160     -64.7459       3.1367  MA1 : 10, MA2 : 5, SL : 0.66%
2023     287.8743          

In [45]:
tb.head()

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month
0,2022-01-10 15:15:00,LONG,2022-01-11 09:15:00,7739.4500,7688.3696,7688.3696,2022-01-18 10:15:00,7928.0500,188.6000,180.7663,2801876.8750,MACD Reversal,15500,12,28.0188,2022,1
1,2022-01-27 14:15:00,LONG,2022-01-27 15:15:00,7360.3500,7311.7717,7556.2150,2022-02-03 14:15:00,7624.9500,264.6000,257.1073,4190849.8050,MACD Reversal,16300,12,41.9085,2022,1
2,2022-02-09 10:15:00,LONG,2022-02-09 11:15:00,7476.7000,7427.3538,7502.8281,2022-02-11 09:15:00,7439.5500,-37.1500,-44.6081,-715960.4062,TSL Hit,16050,12,-7.1596,2022,2
3,2022-02-15 13:15:00,LONG,2022-02-15 14:15:00,7254.4500,7206.5706,7206.5706,2022-02-17 14:15:00,7273.0000,18.5500,11.2863,186787.8513,MACD Reversal,16550,12,1.8679,2022,2
4,2022-02-23 10:15:00,LONG,2022-02-23 11:15:00,7165.2000,7117.9097,7117.9097,2022-02-23 13:15:00,7117.9097,-47.2903,-54.4319,-911733.9036,Initial SL Hit,16750,12,-9.1173,2022,2


In [46]:
tb.to_csv('MACD_midcp.csv')

In [20]:
stats_dictionary = {}
# i = slow_ma , j = fast_ma , k = trailing_ma

sl_pct = [0.006, 0.004]
for i in range(5, 51, 5):
    for j in range(5, 31, 5):
        for k in range(10, 81, 5):
            for sl in sl_pct:
                if i > j:
                    variation = f'MA1 : {i}, MA2 : {j}, SL : {sl * 100}% , TMA : {k}'
                    print(variation)
                    df = macd_logic_from_df(bnf_pandas, i, j)
                    df['ma'] = df['close'].rolling(window=k).mean()
                    new_tb = execute(df, sl)
                    if len(new_tb) > 1:
                        new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                        tradebook_buy_side = new_tb
                        stats = generate_stats(tradebook_buy_side, variation)
                        for overall_roi_dd_ratio, stats_df in stats.items():
                            if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 13:
                                print(stats_df.to_string())
                                stats_dictionary[overall_roi_dd_ratio] = stats_df

MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 10
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 10
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 15
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 15
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 20
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 20
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 25
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 25
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 30
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 30
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 35
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 35
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 40
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 40
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 45
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 45
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 50
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 50
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 55
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 55
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 60
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 60
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 65
MA1 : 10, MA2 : 5, SL : 0.4% , TMA : 65
MA1 : 10, MA2 : 5, SL : 0.6% , TMA : 70


# SENSEX Sorted Stats

In [21]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

{19.661558201742825:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2017      49.8239           46  34.7826                8.4691   
 2018      95.8396           39  41.0256               10.8233   
 2019      50.4024           46  28.2609               11.3312   
 2020     241.6594           45  33.3333               23.6865   
 2021     126.9378           41  24.3902               21.2719   
 2022      67.3508           38  23.6842               20.9643   
 2023      89.3172           40  32.5000               12.5464   
 2024     -12.9557           19  31.5789                4.7312   
 Overall  708.3756          314  31.2102               14.3283   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio  \
 2017                -2.8560     -22.4206       2.2222   
 2018                -3.3623     -25.1719       3.8074   
 2019                -2.9365     -30.2565       1.6658   
 2020                -3.7880     -25.8915       9.3335   
 2021                -2.7671

# BNF Automation Below

In [137]:
symbol = 'midcp'
# symbol = 'nifty'
# symbol = 'fnf'
# symbol = 'midcp'

if symbol == 'bnf':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.0001
elif symbol == 'nifty' or 'fnf':
    LEVERAGE_ = 7
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.0002
elif symbol == 'midcp':
    LEVERAGE_ = 10
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.0005

PORTFOLIO = 1_00_00_000

In [138]:
bnf_pandas = pd.read_csv("../data/midcp_1hr_tv_new.csv")
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

<class 'polars.dataframe.frame.DataFrame'>


In [140]:
bnf_pandas.tail()

,time,open,high,low,close,index,datetime,macd,ma
13376,2024-09-27T11:15:00+05:30,54217.9000,54231.3000,54148.0500,54184.7000,2024-09-27 11:15:00,2024-09-27 11:15:00,144.7985,54168.4933
13377,2024-09-27T12:15:00+05:30,54184.4000,54188.1500,53945.6500,53974.6000,2024-09-27 12:15:00,2024-09-27 12:15:00,109.7174,54165.0367
13378,2024-09-27T13:15:00+05:30,53971.5500,53978.9000,53830.6000,53903.2500,2024-09-27 13:15:00,2024-09-27 13:15:00,74.2038,54156.2533
13379,2024-09-27T14:15:00+05:30,53895.5000,53945.1000,53763.2000,53833.0000,2024-09-27 14:15:00,2024-09-27 14:15:00,38.9756,54140.0800
13380,2024-09-27T15:15:00+05:30,53834.7500,53876.7000,53787.5000,53834.3000,2024-09-27 15:15:00,2024-09-27 15:15:00,11.9610,54121.7867


In [146]:
stats_dictionary = {}
# i = slow_ma , j = fast_ma , k = trailing_ma

sl_pct = [0.02, 0.0175, 0.015, 0.0125, 0.01, 0.0075, 0.0066, 0.005]
for i in range(20, 51, 5):
    for j in range(5, 21, 5):
        for k in range(10, 81, 5):
            for sl in sl_pct:
                variation = f'MA1 : {i}, MA2 : {j}, SL : {sl * 100}% , TMA : {k}'
                print(variation)
                df = macd_logic_from_df(bnf_pandas, i, j)
                df['ma'] = df['close'].rolling(window=k).mean()
                new_tb = execute(df, sl)
                # print(new_tb.tail())
                if len(new_tb) > 1:
                    new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                    tradebook_buy_side = new_tb
                    stats = generate_stats(tradebook_buy_side, variation)
                    for overall_roi_dd_ratio, stats_df in stats.items():
                        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 5:
                            print(stats_df.to_string())
                            stats_dictionary[overall_roi_dd_ratio] = stats_df

MA1 : 20, MA2 : 5, SL : 2.0% , TMA : 10


KeyboardInterrupt: 

# BNF Sorted Stats

In [ ]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

In [142]:
bnf_pandas = pd.read_csv("../data/sensex_min.csv")
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

<class 'polars.dataframe.frame.DataFrame'>


In [1]:
# symbol = 'bnf'
# symbol = 'nifty'
# symbol = 'fnf'
# symbol = 'midcp'
symbol = 'sensex'

if symbol == 'bnf' or symbol == 'bankex':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.0001
elif symbol == 'nifty' or 'fnf':
    LEVERAGE_ = 7
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.0002
elif symbol == 'midcp':
    LEVERAGE_ = 10
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.0005
elif symbol == 'sensex':
    LEVERAGE_ = 5
    LOT_SIZE_ = 10
    SLIPPAGE_ = 0.0001

PORTFOLIO = 1_00_00_000

In [2]:
stats_dictionary = {}
# i = slow_ma , j = fast_ma , k = trailing_ma
tfs = ['15m', '30m', '60m']
sl_pct = [0.015, 0.0125, 0.01, 0.0075, 0.006, 0.005, 0.004]
for i in range(10, 51, 5):
    for j in range(5, 31, 5):
        for k in range(10, 81, 5):
            for tf in tfs:
                for sl in sl_pct:
                    if i > j:
                        if tf == '30m' or tf == '60m':
                            ofs = '15m'
                        else:
                            ofs = '0m'
                        variation = f'MA1 : {i}, MA2 : {j}, SL : {sl * 100}% , TMA : {k} , TF : {tf}'
                        print(variation)
                        df1 = resample(pl.DataFrame(bnf_pandas), tf, ofs)
                        df2 = df1.to_pandas()
                        df = macd_logic_from_df(df2, i, j)
                        df['ma'] = df['close'].rolling(window=k).mean()
                        new_tb = execute(df, sl)
                        if len(new_tb) > 1:
                            new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                            tradebook_buy_side = new_tb
                            stats = generate_stats(tradebook_buy_side, variation)
                            for overall_roi_dd_ratio, stats_df in stats.items():
                                if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 12:
                                    print(stats_df.to_string())
                                    stats_dictionary[overall_roi_dd_ratio] = stats_df

MA1 : 10, MA2 : 5, SL : 1.5% , TMA : 10 , TF : 15m


NameError: name 'resample' is not defined

In [122]:
# tb.tail(50)

In [ ]:
# tb.to_csv('MACD_nifty.csv')